# Reproducing Fig. 1

This notebook exemplifies how to reproduce Figure 1 of the article. The annotations from all screens will be downloaded and parsed to build statistics on phenotypes, which will be displayed using bokeh.

### Dependencies

 * [Matplotlib](http://matplotlib.org/)
 * [NumPy](http://www.numpy.org/)
 * [Pandas](http://pandas.pydata.org/)
 * [Bokeh](https://bokeh.pydata.org/)

In [1]:
import omero
from idr import connection

import numpy as np
import matplotlib.pyplot as plt

from pandas import DataFrame
from pandas import concat
from pandas import read_csv

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.plotting import show
from bokeh.plotting import save
from bokeh.charts import Scatter
from bokeh.models.formatters import TickFormatter
from bokeh.models.formatters import String
from bokeh.models.formatters import List
from bokeh.models.formatters import Dict
from bokeh.models.formatters import Int
from bokeh.models import FixedTicker
from bokeh.models import HoverTool
import bokeh.palettes as bpal

output_notebook()
%matplotlib inline

Loading BokehJS ...

### Method definitions

In [3]:
def getBulkAnnotationAsDf(screenID, conn):
    sc = conn.getObject('Screen', screenID)
    for ann in sc.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            if (ann.getFile().getName() == 'bulk_annotations'):
                if (ann.getFile().getSize()> 1476250900): #about 140Mb?
                    print "that's a big file..."
                    return None
                ofId = ann.getFile().getId()
                break

    original_file = omero.model.OriginalFileI(ofId, False)

    openTable = conn.c.sf.sharedResources().openTable(original_file)
    rowCount = openTable.getNumberOfRows()

    #table to df

    column_names = [col.name for col in openTable.getHeaders()]

    black_list = []
    column_indices = []
    for column_name in column_names:
        if column_name in black_list:
            continue
        column_indices.append(column_names.index(column_name))

    table_data = openTable.slice(column_indices, None)
    data = []
    for index in range(rowCount):
        row_values = [column.values[index] for column in table_data.columns]
        data.append(row_values)

    dfAnn = DataFrame(data)
    dfAnn.columns = column_names
    return dfAnn

In [4]:
def appendPhInfo(phall,screen,df):
    """
    extract all phenotypes information from given bulk annotation file and
    append it to the phall
    """
    phcol = df.columns[[('Phenotype' in s) and ('Term Accession' in s) for s in df.columns ]]
    for s in phcol:
        ph = df[s].unique()
        if ph[0]!='':
            ph = ph[0]            
            desc = df[s.replace('Accession', 'Name')].unique()[0]
        else:
            ph = ph[1]
            desc = df[s.replace('Accession', 'Name')].unique()[1]
        dfph = df[df[s]!='']
        try:
            phall[ph]['n'] = phall[ph]['n']+len(dfph)
            if not (screen in phall[ph]['screens']): 
                phall[ph]['screens'].append(screen)

        except Exception as e: 
            print(e)
            phcur = {'n':len(dfph),'screens':[screen],'desc':desc}
            phall[ph] = phcur

### Connection to the IDR server

In [ ]:
conn = connection()

## Build and display figure

In [9]:
screens= conn.getObjects("Screen")

phall={}
for sc in screens:
    scId = sc.getId()
    name =  sc.getName()
    print 'loading ' + str(scId) + '::' + name
    df = getBulkAnnotationAsDf(scId, conn)
    appendPhInfo(phall, name, df)

loading 1201::idr0015-UNKNOWN-taraoceans/screenA


UnboundLocalError: local variable 'ofId' referenced before assignment

### Disconnect when done loading data

In [ ]:
conn.close()

### Load grouping of phenotypes for ordering and coloring them
Categories were chosen manually offline (see [paper](https://doi.org/10.1101/089359))

In [6]:
dfColor = read_csv('../includes/CMPOAccessionToPhenotypeCategories.csv')
colors = {}
for i,grp in enumerate(dfColor.CmpoPhenotypeCategory.unique()):
    colors[grp] = bpal.Set3_10[i%10]

In [7]:
#add the information to the data and sort it
for ph in phall.keys():
    try:
        phall[ph]['group']=dfColor[dfColor['CmpoAcc']==ph]['CmpoPhenotypeCategory'].values[0]
        phall[ph]['groupColor']=colors[phall[ph]['group']]
        phall[ph]['FigureCmpoName']=dfColor[dfColor['CmpoAcc']==ph]['FigureCmpoName'].values[0]
        
    except:
        print 'pass:'+ph
        del phall[ph]

phalls=sorted(phall.values(), key=lambda x: x['group'])

In [8]:
TOOLS="pan,wheel_zoom,reset"

p = figure(title = "Fig 1", tools=TOOLS,y_axis_type="log",width=1200,toolbar_location="above")


source = ColumnDataSource(
    data=dict(
        ph=[ph['FigureCmpoName'] for ph in phalls],
        n=[ph['n'] for ph in phalls],
        names=[ph['screens'] for ph in phalls],
        desc=[ph['desc'] for ph in phalls],
        x=[2*x for x in range(len(phall.keys()))],
        r=[1*len(ph['screens']) for ph in phalls],
        color=[ph['groupColor'] for ph in phalls],
        groupName=[ph['group'] for ph in phalls]
    ))

label_data = {2*i:x for i,x in enumerate([ph['FigureCmpoName'] for ph in phalls])}

cir=p.circle('x','n',radius='r',source=source,color='color')


hover = HoverTool(
        tooltips=[
            ("Term", "@ph"),
            ("Description", "@desc"),
            ("Number of samples", "@n"),
            ("Screens name", "@names"),
            ("group", "@groupName")
    ]
    )
p.add_tools(hover)

JS_CODE =  """
        _ = require "underscore"
        Model = require "model"
        p = require "core/properties"
        class FixedTickFormatter extends Model
          type: 'FixedTickFormatter'
          doFormat: (ticks) ->
            labels = @get("labels")
            return (labels[tick] ? "" for tick in ticks)
          @define {
            labels: [ p.Any ]
          }
        module.exports =
          Model: FixedTickFormatter
    """

class FixedTickFormatter(TickFormatter):

    labels = Dict(Int, String, help="""
    A mapping of integer ticks values to their labels.
    """)

    __implementation__ = JS_CODE


p.xaxis.formatter = FixedTickFormatter(labels=label_data)
p.xaxis.ticker = FixedTicker(ticks=sorted(label_data.keys()))
p.xaxis.major_label_orientation = np.pi/4.
p.xaxis.axis_label_text_font_size = "10pt"
show(p)
#save(p,'fig1.html')

### License

Copyright (C) 2016 University of Dundee. All Rights Reserved.

This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation; either version 2 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with this program; if not, write to the Free Software Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.